In [1]:
pip install pyqt5


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keyboard

     |████████████████████████████████| 58 kB 1.0 MB/s eta 0:00:01
  Using cached pyobjc_core-7.3-cp38-cp38-macosx_10_9_x86_64.whl (293 kB)


  Using cached pyobjc_framework_Quartz-7.3-cp38-cp38-macosx_10_9_x86_64.whl (143 kB)


  Using cached pyobjc_framework_Cocoa-7.3-cp38-cp38-macosx_10_9_x86_64.whl (273 kB)
  Attempting uninstall: pyobjc-core
    Found existing installation: pyobjc-core 8.5.1
    Uninstalling pyobjc-core-8.5.1:
      Successfully uninstalled pyobjc-core-8.5.1
  Attempting uninstall: pyobjc-framework-Cocoa
    Found existing installation: pyobjc-framework-Cocoa 8.5.1
    Uninstalling pyobjc-framework-Cocoa-8.5.1:
      Successfully uninstalled pyobjc-framework-Cocoa-8.5.1
  Attempting uninstall: pyobjc-framework-Quartz
    Found existing installation: pyobjc-framework-Quartz 8.5.1
    Uninstalling pyobjc-framework-Quartz-8.5.1:
      Successfully uninstalled pyobjc-framework-Quartz-8.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pynput 1.7.6 requires pyobjc-framework-ApplicationServices>=8.0; sys_platform == "darwin", but you have pyobjc-framework-applications

In [2]:
import cv2
import numpy as np
import face_recognition as fr
import os
import datetime

In [3]:
class Face_Detection:
    def __init__(self):
        path = "images"

        images = []
        classNames = []
        mylst = os.listdir(path)
        for cls in range(1,len(mylst)):
            if mylst[cls]!=".ipynb_checkpoints":
                curren = cv2.imread(f"{path}/{mylst[cls]}")
                images.append(curren)
                x = mylst[cls].split(".")
                if x[0]!='':
                    classNames.append(x[0])
        print(classNames)
        def find_encodings(lst_images):
            encode_lst = []
            for img in lst_images:
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                l = fr.face_locations(img)
                encoding = fr.face_encodings(img,l)[0]
                encode_lst.append(encoding)
            #print(encode_lst)
            return encode_lst

        encodeListForKnownFaces=find_encodings(images)
        print("Encoding Completed")

        cam = cv2.VideoCapture(0)
        att = [] #Attendence
        f1 = open("Attendence.txt","w",buffering = 1)
        while True:
            ret,frame = cam.read() #ret = garbage error

            imgs = cv2.resize(frame,(0,0),None,0.25,0.25)
            imgs = cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB)
            location = fr.face_locations(imgs)
            encoding = fr.face_encodings(imgs,location)

            for encode,faceloc in zip(encoding,location):
                matches = fr.compare_faces(encodeListForKnownFaces,encode)

                faceDist = fr.face_distance(encodeListForKnownFaces,encode)
                #print(faceDist)

                matchIndex = np.argmin(faceDist)
                #print(matchIndex)

                if matches[matchIndex]:
                    name = classNames[matchIndex].upper()
                    print(name)
                    if name not in att:
                        att.append(name)
                        f1.write(f"{name} - {datetime.datetime.now()}\n")
                        print(att)


                    y1,x2,y2,x1 = faceloc
                    y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                    center = ((x1+x2)//2,(y1+y2)//2)


                    (h,w) = frame.shape[:2] #resolution of the frame
                    center_frame = ((0+w)//2,(0+h)//2)

                    #rectangle for the face frame
                    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),4)
                    cv2.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                    cv2.putText(frame,f"{name}",(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

                    #line on the face frame
                    #cv2.line(frame,(x1,y1),(x2,y2),(255,0,255),2)
                    #cv2.line(frame,(x2,y1),(x1,y2),(255,0,255),2)

                    #rectangle for the center frame
                    cv2.rectangle(frame,(center_frame[0]+145,center_frame[1]+145),(center_frame[0]-145,center_frame[1]-145),(0,255,0),4)

                    #Center of the Face
                    cv2.putText(frame,f"Face : {center}",(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

                    #Axis lines
                    cv2.line(frame,(center_frame[0],0),(center_frame[0],h),(144, 238, 144),5)
                    cv2.line(frame,(0,center_frame[1]),(w,center_frame[1]),(144, 238, 144),5)

                    #Center of the frame
                    cv2.putText(frame,f"center : {(center_frame[0],center_frame[1])}",(50,100),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

                    #Navigation Statements
                    if center[0]<center_frame[0]:
                        if center[1]<center_frame[1]: 
                            cv2.putText(frame,f"Go Right {center_frame[0]-center[0]} pixels and go Down {center_frame[1]-center[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                        else:
                            cv2.putText(frame,f"Go Right {center_frame[0]-center[0]} pixels and go Up {center[1]-center_frame[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                    elif center[0]>center_frame[0]:
                        if center[1]<center_frame[1]:
                            cv2.putText(frame,f"Go Left {center[0]-center_frame[0]} pixels and go Down {center_frame[1]-center[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                        else:
                            cv2.putText(frame,f"Go Left {center[0]-center_frame[0]} pixels and go Up {center[1]-center_frame[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)




            cv2.imshow("Webcame",frame)
            key = cv2.waitKey(1)
            if key == 27:
                cam.release()
                cv2.destroyAllWindows()
                break 
        f1.close()



In [4]:
import PyQt5
import sys
from PyQt5.QtCore import pyqtSlot, QRect
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QTableWidget, QTableWidgetItem, QHeaderView
from PyQt5.QtGui import QPixmap
import xlsxwriter as xl
import keyboard

In [ ]:
import PyQt5
import sys
from PyQt5.QtCore import pyqtSlot, QRect
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QTableWidget, QTableWidgetItem, QHeaderView
from PyQt5.QtGui import QPixmap
import xlsxwriter as xl
import keyboard







class App(QWidget):
    def __init__(self):
        super().__init__()
        
        #Initializing the parameters
        self.title = "Attendence"
        self.left = 500
        self.top = 200
        self.width = 1080
        self.height = 720
        self.initUI()
        
        
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left,self.top,self.width,self.height)
        
        
        vbox = QVBoxLayout()
        
        #Setting Background Image
        labelImage = QLabel(self)
        pixmap = QPixmap("bg3.jpeg")
        labelImage.setPixmap(pixmap)
        
        vbox.addWidget(labelImage)
        self.setLayout(vbox)
        
        
        
        
        
        #Button2 declaration and design
        self.button2 = QPushButton("Stop Attendence",self)
        self.button2.setGeometry(QRect(300,360,150,50))
        self.button2.setToolTip("Will Create an Excel File")
                
        self.button2.setStyleSheet("QPushButton"
                             "{"
                             "background-color : lightblue;"
                             "}"
                             "QPushButton::hover"
                             "{"
                             "background-color : red;"
                             "}"
                             )
        self.button2.clicked.connect(self.off_cam)
        

        
        #Button1 declaration and design
        self.button = QPushButton("Start Attendence",self)
        self.button.setToolTip("Will Turn On The Camera")
        self.button.setGeometry(QRect(700,360,150,50))
        self.button.setStyleSheet("QPushButton"
                             "{"
                             "background-color : lightblue;"
                             "}"
                             "QPushButton::hover"
                             "{"
                             "background-color : green;"
                             "}")
        self.button.clicked.connect(self.on_cam)

        self.show()


    @pyqtSlot()

    def on_cam(self):
        #Calling the face detection class
        camera = Face_Detection()
        
        #camera()
        
    def off_cam(self):
        
        #Keyboard will simulate "esc" for the face detection to turn of
        keyboard.send("esc")
        
        #calling the write_excel method
        self.write_excel()
        
    
    def write_excel(self):
        
        #Writing on excel file
        
        f = open("Attendence.txt","r",buffering = 1)

        d = [i.strip() for i in f.readlines()]
        lst  = []
        for i in d:
            x = i.split()
            a = x[0]
            b = x[2]
            c = x[3].split(".")
            temp = [a,b,c[0]]
            lst.append(tuple(temp))

        workbook = xl.Workbook("Attendence.xlsx")
        worksheet = workbook.add_worksheet()
        worksheet.write("A1","Names")
        worksheet.write("B1","Dates")
        worksheet.write("C1","Time")
        idx = 2
        for att in lst:
            worksheet.write(f"A{idx}",att[0])
            worksheet.write(f"B{idx}",att[1])
            worksheet.write(f"C{idx}",att[2])
            idx+=1

        f.close()
        workbook.close()
        
        
    

if __name__ == "__main__":
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())
    

['Tausif', 'Goni', 'shafin', 'shamma', 'abrar', 'Wasikursaki']
Encoding Completed
WASIKURSAKI
['WASIKURSAKI']
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WA